In [1]:
import scanpy
import json
import pandas as pd
import scanpy as sc

In [2]:
DEGs_PATH = "/orcd/data/omarabu/001/Omnicell_datasets/DEGs_per_pert_satija_IFNB_raw.json"

In [3]:
DEGs = json.load(open(DEGs_PATH))


p_threshold = 0.05
cell_types = sorted(list(DEGs.keys()))
perts = sorted(list(DEGs["A549"].keys()))


In [4]:
for cell in DEGs:
    for pert in DEGs[cell]:
        df = pd.DataFrame.from_dict(DEGs[cell][pert], orient='index')
        df = df[df['pvals_adj'] < p_threshold]
        DEGs[cell][pert] = df


avg_number_DEGs_pert_pert = {}
for p in perts:
    number_DEGs = []
    for cell in cell_types:
        if p in DEGs[cell]:
            number_DEGs.append(len(DEGs[cell][p]))

    avg_number_DEGs_pert_pert[p] = sum(number_DEGs) / len(number_DEGs)

avg_number_DEGs_pert_pert

{'ADAR': 1001.6666666666666,
 'AKT1': 70.0,
 'BATF2': 69.5,
 'BRD9': 71.33333333333333,
 'CEBPB': 289.0,
 'CEBPG': 195.16666666666666,
 'CRKL': 45.833333333333336,
 'DRAP1': 427.1666666666667,
 'DTX3L': 76.66666666666667,
 'ELK1': 150.16666666666666,
 'ETS1': 157.5,
 'ETS2': 155.16666666666666,
 'FOS': 28.0,
 'H1F0': 96.66666666666667,
 'HERC5': 138.5,
 'HERC6': 135.66666666666666,
 'HES4': 129.66666666666666,
 'ID1': 378.0,
 'ID2': 366.1666666666667,
 'ID3': 50.666666666666664,
 'IFI16': 118.5,
 'IFNAR1': 616.1666666666666,
 'IFNAR2': 1233.0,
 'IRF1': 283.8333333333333,
 'IRF3': 73.83333333333333,
 'IRF7': 122.66666666666667,
 'IRF9': 205.0,
 'JAK1': 544.0,
 'JUN': 148.16666666666666,
 'MAP3K14': 68.66666666666667,
 'MAPK14': 184.33333333333334,
 'MAPK8': 248.5,
 'MEF2A': 47.666666666666664,
 'MYC': 98.5,
 'NFE2L3': 176.33333333333334,
 'NFKB1': 78.83333333333333,
 'PARP12': 63.166666666666664,
 'POU2F1': 115.66666666666667,
 'RAP1GAP': 53.333333333333336,
 'RAPGEF1': 63.1666666666666

In [ ]:

perturbative_perts = [p for p in perts if (avg_number_DEGs_pert_pert[p] > 400 and p != "ADAR")]
perturbative_perts

unperturbative_perts = [p for p in perts if p not in perturbative_perts]
len(unperturbative_perts)

52

In [ ]:
NUMBER_DEGs_COMPARAISON = 20

avg_overlaps_dict = {}
overlaps_dict = {}
for i in range(len(perts)):
    p1 = perts[i]
    for j in range(i+1, len(perts)):
        p2 = perts[j]
        if p1 != p2:
            overlaps = []
            overlaps_per_cell = {}
            for cell in cell_types:
                if p1 in DEGs[cell] and p2 in DEGs[cell]:
                    overlap = len(DEGs[cell][p1].index[:NUMBER_DEGs_COMPARAISON].intersection(DEGs[cell][p2].index[:NUMBER_DEGs_COMPARAISON]))
                    overlaps.append(overlap)
                    overlaps_per_cell[cell] = overlap
            
            
            overlaps_dict[(p1, p2)] = overlaps_per_cell
            mean_overlap = sum(overlaps) / len(overlaps)

            avg_overlaps_dict[(p1, p2)] = mean_overlap

        

In [ ]:
sorted_avg_overlaps = sorted(avg_overlaps_dict.items(), key=lambda x: x[1], reverse=True)
sorted_avg_overlaps


[(('IFNAR1', 'IFNAR2'), 17.666666666666668),
 (('IFNAR1', 'STAT2'), 17.333333333333332),
 (('IFNAR2', 'STAT2'), 17.166666666666668),
 (('IFNAR2', 'TYK2'), 17.0),
 (('IFNAR1', 'TYK2'), 16.666666666666668),
 (('STAT2', 'TYK2'), 16.5),
 (('IFNAR2', 'JAK1'), 11.666666666666666),
 (('IFNAR1', 'JAK1'), 11.5),
 (('JAK1', 'STAT2'), 11.5),
 (('JAK1', 'TYK2'), 10.833333333333334),
 (('IFNAR1', 'STAT1'), 10.666666666666666),
 (('IFNAR2', 'STAT1'), 10.333333333333334),
 (('STAT1', 'STAT2'), 10.333333333333334),
 (('STAT1', 'TYK2'), 9.666666666666666),
 (('JAK1', 'STAT1'), 9.5),
 (('IFNAR1', 'IRF9'), 8.833333333333334),
 (('IFNAR2', 'IRF9'), 8.833333333333334),
 (('IRF9', 'STAT2'), 8.833333333333334),
 (('IRF9', 'TYK2'), 8.833333333333334),
 (('TYK2', 'USP18'), 8.666666666666666),
 (('IRF9', 'JAK1'), 7.333333333333333),
 (('STAT2', 'USP18'), 7.333333333333333),
 (('IFNAR1', 'USP18'), 7.166666666666667),
 (('IFNAR2', 'USP18'), 7.166666666666667),
 (('IRF9', 'STAT1'), 6.833333333333333),
 (('JAK1', '

In [ ]:
overlaps_dict[('IFNAR1', 'TYK2')]

{'A549': 16, 'BXPC3': 17, 'HAP1': 19, 'HT29': 18, 'K562': 15, 'MCF7': 15}

In [ ]:
from omnicell.data.catalogue import Catalogue


In [ ]:
catalogue = Catalogue("./configs/catalogue")

In [ ]:
catalogue.get_dataset_details("satija_IFNB_raw")

DatasetDetails(path='/orcd/data/omarabu/001/Omnicell_datasets/satija_IFNB_raw/Seurat_IFNB.h5ad', folder_path='/orcd/data/omarabu/001/Omnicell_datasets/satija_IFNB_raw/', cell_key='cell_type', control='NT', pert_key='gene', var_names_key='gene', HVG=False, log1p_transformed=False, count_normalized=False, description='Some description of the dataset', pert_embeddings=['UCE_ESM2_Mean_Imputed', 'BioBERT'], cell_embeddings=[])

In [ ]:
dd = catalogue.get_dataset_details("satija_IFNB_raw")
adata = sc.read(dd.path)


In [29]:
adata.obs[dd.cell_key].value_counts()

cell_type
BXPC3    84057
HAP1     67959
MCF7     58903
HT29     56836
A549     32233
K562     28554
Name: count, dtype: int64

In [13]:
across_cells_overlaps = {}
for c1 in cell_types:
    for c2 in cell_types:
        for p1 in perturbative_perts:
            for p2 in perturbative_perts:
                if p1 != p2:
                    if p1 in DEGs[c1] and p2 in DEGs[c2]:
                        overlap = len(DEGs[c1][p1].index[:NUMBER_DEGs_COMPARAISON].intersection(DEGs[c2][p2].index[:NUMBER_DEGs_COMPARAISON]))
                        across_cells_overlaps[((c1, p1), (c2, p2))] = overlap

across_cells_overlaps

{(('A549', 'DRAP1'), ('A549', 'IFNAR1')): 1,
 (('A549', 'DRAP1'), ('A549', 'IFNAR2')): 1,
 (('A549', 'DRAP1'), ('A549', 'JAK1')): 0,
 (('A549', 'DRAP1'), ('A549', 'STAT1')): 0,
 (('A549', 'DRAP1'), ('A549', 'STAT2')): 1,
 (('A549', 'DRAP1'), ('A549', 'TYK2')): 1,
 (('A549', 'DRAP1'), ('A549', 'USP18')): 0,
 (('A549', 'DRAP1'), ('A549', 'VAV1')): 0,
 (('A549', 'IFNAR1'), ('A549', 'DRAP1')): 1,
 (('A549', 'IFNAR1'), ('A549', 'IFNAR2')): 18,
 (('A549', 'IFNAR1'), ('A549', 'JAK1')): 5,
 (('A549', 'IFNAR1'), ('A549', 'STAT1')): 1,
 (('A549', 'IFNAR1'), ('A549', 'STAT2')): 14,
 (('A549', 'IFNAR1'), ('A549', 'TYK2')): 16,
 (('A549', 'IFNAR1'), ('A549', 'USP18')): 6,
 (('A549', 'IFNAR1'), ('A549', 'VAV1')): 0,
 (('A549', 'IFNAR2'), ('A549', 'DRAP1')): 1,
 (('A549', 'IFNAR2'), ('A549', 'IFNAR1')): 18,
 (('A549', 'IFNAR2'), ('A549', 'JAK1')): 5,
 (('A549', 'IFNAR2'), ('A549', 'STAT1')): 1,
 (('A549', 'IFNAR2'), ('A549', 'STAT2')): 14,
 (('A549', 'IFNAR2'), ('A549', 'TYK2')): 18,
 (('A549', 'IFNA

In [14]:

across_cells_overlaps_all = {}
for c1 in cell_types:
    for c2 in cell_types:
        for p1 in perts:
            for p2 in perts:
                if p1 != p2:
                    if p1 in DEGs[c1] and p2 in DEGs[c2]:
                        overlap = len(DEGs[c1][p1].index[:NUMBER_DEGs_COMPARAISON].intersection(DEGs[c2][p2].index[:NUMBER_DEGs_COMPARAISON]))
                        across_cells_overlaps_all[((c1, p1), (c2, p2))] = overlap

unperturbative_DEGs = []

for key, DEG in across_cells_overlaps_all.items():
    (c1, p1), (c2, p2) = key 
    if p1 in unperturbative_perts and p2 in unperturbative_perts:
        unperturbative_DEGs.append([key, DEG])

sorted(unperturbative_DEGs, key=lambda x: x[1], reverse=True)

[[(('MCF7', 'ETS2'), ('MCF7', 'MAPK8')), 15],
 [(('MCF7', 'MAPK8'), ('MCF7', 'ETS2')), 15],
 [(('HT29', 'ETS2'), ('HT29', 'SP110')), 13],
 [(('HT29', 'SP110'), ('HT29', 'ETS2')), 13],
 [(('MCF7', 'ELK1'), ('MCF7', 'ETS2')), 13],
 [(('MCF7', 'ETS2'), ('MCF7', 'ELK1')), 13],
 [(('MCF7', 'ELK1'), ('MCF7', 'MAPK8')), 12],
 [(('MCF7', 'MAPK8'), ('MCF7', 'ELK1')), 12],
 [(('MCF7', 'BATF2'), ('MCF7', 'ELK1')), 10],
 [(('MCF7', 'CEBPB'), ('MCF7', 'ETS2')), 10],
 [(('MCF7', 'ELK1'), ('MCF7', 'BATF2')), 10],
 [(('MCF7', 'ETS2'), ('MCF7', 'CEBPB')), 10],
 [(('MCF7', 'BATF2'), ('MCF7', 'ETS2')), 9],
 [(('MCF7', 'CEBPB'), ('MCF7', 'ELK1')), 9],
 [(('MCF7', 'CEBPB'), ('MCF7', 'ID3')), 9],
 [(('MCF7', 'CEBPB'), ('MCF7', 'MAPK8')), 9],
 [(('MCF7', 'ELK1'), ('MCF7', 'CEBPB')), 9],
 [(('MCF7', 'ETS2'), ('MCF7', 'BATF2')), 9],
 [(('MCF7', 'ID3'), ('MCF7', 'CEBPB')), 9],
 [(('MCF7', 'ID3'), ('MCF7', 'MAPK8')), 9],
 [(('MCF7', 'MAPK8'), ('MCF7', 'CEBPB')), 9],
 [(('MCF7', 'MAPK8'), ('MCF7', 'ID3')), 9],
 [

In [ ]:
nearest_neighbor_per_cell_type_and_pert = {}
for cell in cell_types:
    for pert in perturbative_perts:
        for other_cell in cell_types:
            if cell == other_cell:
                overlaps = []
                for neighbor_pert in perturbative_perts:
                    if pert != neighbor_pert:
                        overlap = across_cells_overlaps[((cell, pert), (other_cell, neighbor_pert))]
                        overlaps.append([other_cell, neighbor_pert, overlap])

                nearest_neighbor_per_cell_type_and_pert[(cell, pert, other_cell)] = sorted(overlaps, key=lambda x: x[2], reverse=True)

nearest_neighbor_per_cell_type_and_pert

sorted(list(nearest_neighbor_per_cell_type_and_pert.items()), key=lambda x: x[0][1])
                    

[(('A549', 'DRAP1', 'A549'),
  [['A549', 'IFNAR1', 1],
   ['A549', 'IFNAR2', 1],
   ['A549', 'STAT2', 1],
   ['A549', 'TYK2', 1],
   ['A549', 'JAK1', 0],
   ['A549', 'STAT1', 0],
   ['A549', 'USP18', 0],
   ['A549', 'VAV1', 0]]),
 (('BXPC3', 'DRAP1', 'BXPC3'),
  [['BXPC3', 'JAK1', 1],
   ['BXPC3', 'STAT1', 1],
   ['BXPC3', 'VAV1', 1],
   ['BXPC3', 'IFNAR1', 0],
   ['BXPC3', 'IFNAR2', 0],
   ['BXPC3', 'STAT2', 0],
   ['BXPC3', 'TYK2', 0],
   ['BXPC3', 'USP18', 0]]),
 (('HAP1', 'DRAP1', 'HAP1'),
  [['HAP1', 'IFNAR1', 0],
   ['HAP1', 'IFNAR2', 0],
   ['HAP1', 'JAK1', 0],
   ['HAP1', 'STAT1', 0],
   ['HAP1', 'STAT2', 0],
   ['HAP1', 'TYK2', 0],
   ['HAP1', 'USP18', 0],
   ['HAP1', 'VAV1', 0]]),
 (('HT29', 'DRAP1', 'HT29'),
  [['HT29', 'IFNAR1', 1],
   ['HT29', 'IFNAR2', 1],
   ['HT29', 'JAK1', 1],
   ['HT29', 'STAT1', 1],
   ['HT29', 'STAT2', 1],
   ['HT29', 'TYK2', 1],
   ['HT29', 'USP18', 1],
   ['HT29', 'VAV1', 0]]),
 (('K562', 'DRAP1', 'K562'),
  [['K562', 'IFNAR1', 0],
   ['K562', 'IF

In [36]:
heldout_perturbative_perts = ["IFNAR2", "STAT2"]
holdout_cell = ["MCF7"]

included_perturbative_perts = [p for p in perturbative_perts if p not in heldout_perturbative_perts]
included_perturbative_perts


['DRAP1', 'IFNAR1', 'JAK1', 'STAT1', 'TYK2', 'USP18', 'VAV1']

In [24]:
# Holdout 20 unpert perts


# Tasks 
# Across Perts Neighbor 
# [A549, IFNAR2] [A549, STAT2] + Unperturbative heldout Perts



# Across Cells 
# [MCF7, USP18], [MCF7, IFNAR2], [MCF7, STAT2] + Unperturbative heldout + Unperturbative Included (10)

In [39]:
#Randomly select 20 unperturbative perts 
import random
random.seed(0)
heldout_unperturbative_perts = random.sample(unperturbative_perts, 10)
unpertubative_included = [p for p in unperturbative_perts if p not in heldout_unperturbative_perts]
unpert_included_eval = random.sample(unpertubative_included, 5)

heldout_perts = heldout_unperturbative_perts + heldout_perturbative_perts

In [26]:
heldout_pert_mask = adata.obs[dd.pert_key].isin(heldout_perts)
heldout_cell_mask = (adata.obs[dd.cell_key].isin(holdout_cell) & ~(adata.obs[dd.pert_key]=="NT"))

adata_train = adata[~heldout_pert_mask & ~heldout_cell_mask]
adata_train
adata_train.obs[dd.pert_key].value_counts()

adata_train.obs[adata_train.obs[dd.cell_key] == "MCF7"]

,orig.ident,nCount_RNA,nFeature_RNA,sample,bc1_well,bc2_well,bc3_well,percent.mito,cell_type,pathway,RNA_snn_res.0.9,seurat_clusters,sample_ID,Batch_info,guide,gene,mixscale_score
10_91_31_1_1_1_1_1_1_1_1_1,10,9998,3931,MCF7_IFNB,A10,H7,C7,5.461092,MCF7,IFNB,8.0,8.0,sample_1,Rep1,NTg4,NT,0.0
11_50_93_1_1_1_1_1_1_1_1_1,11,9949,4046,MCF7_IFNB,A11,E2,H9,3.658659,MCF7,IFNB,8.0,8.0,sample_1,Rep1,NTg11,NT,0.0
11_93_12_1_1_1_1_1_1_1_1_1,11,9698,4281,MCF7_IFNB,A11,H9,A12,2.907816,MCF7,IFNB,8.0,8.0,sample_1,Rep1,NTg6,NT,0.0
12_18_81_1_1_1_1_1_1_1_1_1,12,9621,3958,MCF7_IFNB,A12,B6,G9,1.268059,MCF7,IFNB,8.0,8.0,sample_1,Rep1,NTg13,NT,0.0
09_49_95_1_1_1_1_1_1_1_1_1,9,9476,4183,MCF7_IFNB,A9,E1,H11,3.060363,MCF7,IFNB,8.0,8.0,sample_1,Rep1,NTg9,NT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11_60_02_2_2,11,3379,2005,MCF7_IFNB,A11,E12,A2,5.622965,MCF7,IFNB,NaN,NaN,sample_16,Rep2,NTg4,NT,0.0
12_37_18_2_2,12,3195,1809,MCF7_IFNB,A12,D1,B6,4.538341,MCF7,IFNB,NaN,NaN,sample_16,Rep2,NTg13,NT,0.0
10_31_02_2_2,10,3130,1736,MCF7_IFNB,A10,C7,A2,4.984026,MCF7,IFNB,NaN,NaN,sample_16,Rep2,NTg13,NT,0.0
12_18_26_2_2,12,3017,1892,MCF7_IFNB,A12,B6,C2,2.850514,MCF7,IFNB,NaN,NaN,sample_16,Rep2,NTg6,NT,0.0


In [35]:
adata_train.obs[(adata_train.obs[dd.cell_key] == "A549") & (adata.obs[dd.pert_key]=="NT")]

/tmp/ipykernel_163200/3117941990.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata_train.obs[(adata_train.obs[dd.cell_key] == "A549") & (adata.obs[dd.pert_key]=="NT")]


,orig.ident,nCount_RNA,nFeature_RNA,sample,bc1_well,bc2_well,bc3_well,percent.mito,cell_type,pathway,RNA_snn_res.0.9,seurat_clusters,sample_ID,Batch_info,guide,gene,mixscale_score
06_28_67_1_1_1_1_1_1_1_1_1,6,8999,3854,A549_IFNB,A6,C4,F7,9.189910,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg8,NT,0.0
05_69_22_1_1_1_1_1_1_1_1_1,5,6291,3106,A549_IFNB,A5,F9,B10,3.163249,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg9,NT,0.0
08_42_41_1_1_1_1_1_1_1_1_1,8,5107,2567,A549_IFNB,A8,D6,D5,7.538672,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg11,NT,0.0
06_01_93_1_1_1_1_1_1_1_1_1,6,4723,2552,A549_IFNB,A6,A1,H9,3.726445,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg1,NT,0.0
07_27_61_1_1_1_1_1_1_1_1_1,7,4683,2617,A549_IFNB,A7,C3,F1,1.281230,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg11,NT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05_15_40_2_2,5,3747,2112,A549_IFNB,A5,B3,D4,0.693888,A549,IFNB,NaN,NaN,sample_16,Rep2,NTg1,NT,0.0
07_88_53_2_2,7,3580,2168,A549_IFNB,A7,H4,E5,0.865922,A549,IFNB,NaN,NaN,sample_16,Rep2,NTg9,NT,0.0
05_35_76_2_2,5,3476,2083,A549_IFNB,A5,C11,G4,1.323360,A549,IFNB,NaN,NaN,sample_16,Rep2,NTg14,NT,0.0
05_70_76_2_2,5,3367,2015,A549_IFNB,A5,F10,G4,4.959905,A549,IFNB,NaN,NaN,sample_16,Rep2,NTg9,NT,0.0


In [27]:
train_cell_labels = list(adata_train.obs[dd.cell_key])
train_pert_labels = list(adata_train.obs[dd.pert_key])
genes = list(adata_train.var['gene'])
print(train_cell_labels[:10])
print(train_pert_labels[:10])
print(genes[:10])

['A549', 'A549', 'A549', 'A549', 'A549', 'A549', 'A549', 'A549', 'A549', 'A549']
['TRAFD1', 'HES4', 'NT', 'STAT5A', 'STAT4', 'RNF114', 'UBE2L6', 'MAPK8', 'ETS1', 'FOS']
['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2', 'GCLC', 'NFYA']


In [ ]:
#Tasks on A549
print(heldout_perturbative_perts)
print(heldout_unperturbative_perts)
print(heldout_perturbative_perts + heldout_unperturbative_perts)



['IFNAR2', 'STAT2']
['JUN', 'UBA7', 'MAPK14', 'BATF2', 'ID1', 'PARP12', 'NFKB1', 'MAP3K14', 'IFI16', 'NFE2L3']
['IFNAR2', 'STAT2', 'JUN', 'UBA7', 'MAPK14', 'BATF2', 'ID1', 'PARP12', 'NFKB1', 'MAP3K14', 'IFI16', 'NFE2L3']


In [45]:
#Tasks on MCF7
print(heldout_perturbative_perts)
print(heldout_unperturbative_perts[:5])
print(included_perturbative_perts)
print(unpert_included_eval)

print(heldout_perturbative_perts + heldout_unperturbative_perts[:5] + included_perturbative_perts + unpert_included_eval)


['IFNAR2', 'STAT2']
['JUN', 'UBA7', 'MAPK14', 'BATF2', 'ID1']
['DRAP1', 'IFNAR1', 'JAK1', 'STAT1', 'TYK2', 'USP18', 'VAV1']
['MEF2A', 'TRIM21', 'HERC6', 'STAT3', 'ETS1']
['IFNAR2', 'STAT2', 'JUN', 'UBA7', 'MAPK14', 'BATF2', 'ID1', 'DRAP1', 'IFNAR1', 'JAK1', 'STAT1', 'TYK2', 'USP18', 'VAV1', 'MEF2A', 'TRIM21', 'HERC6', 'STAT3', 'ETS1']


In [31]:
from scipy import sparse
import numpy as np

# Assuming you have a CSC matrix called 'csc_matrix'
# You can convert it to COO format using .tocoo()
coo_train = adata_train.X.tocoo()

In [32]:
import os
os.makedirs("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks", exist_ok=True)

In [33]:
#Saving the training data to the new directory
import scipy
scipy.sparse.save_npz("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train.npz", coo_train)
import numpy as np
np.savez("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train_cell_labels.npz", train_cell_labels)
np.savez("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train_pert_labels.npz", train_pert_labels)
np.savez("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train_genes.npz", genes)

In [46]:
train = scipy.sparse.load_npz("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train.npz")
train_cell_labels = np.load("/orcd/data/omarabu/001/Omnicell_datasets/deepmind_tasks/task1_train_cell_labels.npz", allow_pickle=True)

In [50]:
train

<228825x34025 sparse matrix of type '<class 'numpy.int64'>'
	with 652268963 stored elements in COOrdinate format>